In [ ]:
#Gavin Hunsinger

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

In [2]:
slice_name = 'DNS_Server_Infra'

site1 = "SALT"
base_cores=2
base_ram=8
base_disk=10
desired_image='default_ubuntu_22'

node1_name = 'ntp-make'
node2_name = 'ntp-client'
router1_name = 'R1'

network1_name='server_side' 
network2_name='client_side'

nic_model = 'NIC_Basic'

node1_nic_name = 'NODE1NIC'
node2_nic_name = 'NODE2NIC'
node3_nic_name = 'NODE3NIC'

R1_nic1_name = 'nicR1a' #R1 interfaces
R1_nic2_name = 'nicR1b'

In [3]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

In [4]:
#network definitions
#subnets and gateways used for routing configurations
net1_subnet = IPv4Network("10.10.10.0/24")
net1 = slice.add_l2network(name=network1_name, subnet=net1_subnet)

net2_subnet = IPv4Network("192.168.47.0/24")
net2 = slice.add_l2network(name=network2_name, subnet=net2_subnet)

In [5]:
R1 = slice.add_node(name=router1_name, site=site1, cores=base_cores, ram=base_ram, disk=base_disk, image=desired_image)
R1_iface1 = R1.add_component(model=nic_model, name=R1_nic1_name).get_interfaces()[0]
R1_iface2 = R1.add_component(model=nic_model, name=R1_nic2_name).get_interfaces()[0]

net1.add_interface(R1_iface1)
net2.add_interface(R1_iface2)

In [6]:
R1.add_route(subnet=net2_subnet, next_hop=net1.get_gateway())
R1.add_route(subnet=net1_subnet, next_hop=net2.get_gateway())

In [7]:
#server
N1 = slice.add_node(name=node1_name, site=site1, cores=base_cores, ram=base_ram, disk=base_disk, image=desired_image)
N1_Nic =  N1.add_component(model=nic_model, name=node1_nic_name).get_interfaces()[0]
net2.add_interface(N1_Nic)

In [8]:
#client
N2 = slice.add_node(name=node2_name, site=site1, cores=base_cores, ram=base_ram, disk=base_disk, image=desired_image)
N2_Nic = N2.add_component(model= nic_model, name=node2_nic_name).get_interfaces()[0]
net2.add_interface(N2_Nic)

In [9]:
N1.add_route(subnet=net1_subnet, next_hop=net2.get_gateway())

N2.add_route(subnet=net1_subnet, next_hop=net2.get_gateway())

In [10]:
slice.submit();

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f7e0cf76510>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
#client and server on 192.168.47.0/24
N1 = slice.get_node(name=node1_name)
nic1 = N1.get_interface(network_name=network2_name)
nic1.ip_addr_add(IPv4Address('192.168.47.101'), net2_subnet)

#client
N2 = slice.get_node(name=node2_name)
nic2 = N2.get_interface(network_name=network2_name)
nic2.ip_addr_add(IPv4Address('192.168.47.50'), net2_subnet)

In [ ]:
#router
R1 = slice.get_node(name=router1_name)
R1nic1 = R1.get_interface(network_name =network1_name)
R1nic2 = R1.get_interface(network_name =network2_name)
R1nic1.ip_addr_add(IPv4Address('10.10.10.254'), net1_subnet)
R1nic2.ip_addr_add(IPv4Address('192.168.47.254'), net2_subnet)

In [ ]:
#router's routing table
stdout, stderr = R1.execute(f'sudo ip link set enp7s0 up')
stdout, stderr = R1.execute(f'sudo ip link set enp8s0 up')
stdout, stderr = R1.execute(f'sudo sysctl net.ipv4.ip_forward=1')

stdout, stderr = R1.execute(f'sudo ufw status verbose')

In [ ]:
#server routing table
stdout, stderr = N1.execute(f'sudo ip link set enp7s0 up')
stdout, stderr = N1.execute(f'sudo ip route add 10.10.10.0/24 via 192.168.47.254')
stdout, stderr = N1.execute(f'sudo ufw status verbose')

In [ ]:
#client routing table
stdout, stderr = N2.execute(f'sudo ip link set enp7s0 up')
stdout, stderr = N2.execute(f'sudo ip route add 10.10.10.0/24 via 192.168.47.254')
stdout, stderr = N2.execute(f'sudo ufw status verbose')

In [ ]:
stdout, stderr = N1.execute(f'sudo apt-get -y update')

In [ ]:
stdout, stderr = N2.execute(f'sudo apt-get -y update')

In [ ]:
stdout, stderr = N1.execute(f'sudo apt-get install net-tools bind9 bind9utils bind9-doc dnsutils git -y')

In [ ]:
stdout, stderr = N2.execute(f'sudo apt-get install dnsutils net-tools git -y')

In [ ]:
#dns server setup
stdout, stderr = N1.execute(f'git clone https://github.com/imgavinhi/RIT_Network_Research')

In [ ]:
#starting and enabling dns service
stdout, stderr = N1.execute(f'sudo systemctl enable --now bind9')

In [ ]:
#copying dns config files where they should go
stdout, stderr = N1.execute(f'sudo mkdir -p /etc/bind/zones')
stdout, stderr = N1.execute(f'sudo cp /home/ubuntu/RIT_Network_Research/Jupyter_Fabric_Testbed_Infrastructure/DNS_Infra/named.conf.options /etc/bind/named.conf.options')
stdout, stderr = N1.execute(f'sudo cp /home/ubuntu/RIT_Network_Research/Jupyter_Fabric_Testbed_Infrastructure/DNS_Infra/named.conf.local /etc/bind/named.conf.local')
stdout, stderr = N1.execute(f'sudo cp /home/ubuntu/RIT_Network_Research/Jupyter_Fabric_Testbed_Infrastructure/DNS_Infra/research.net /etc/bind/zones/research.net')
stdout, stderr = N1.execute(f'sudo cp /home/ubuntu/RIT_Network_Research/Jupyter_Fabric_Testbed_Infrastructure/DNS_Infra/research.net.rev /etc/bind/zones/research.net.rev')

In [ ]:
#applying server config
stdout, stderr = N1.execute(f'sudo systemctl restart bind9')

In [ ]:
#checking server status
stdout, stderr = N1.execute(f'sudo systemctl status bind9 --no-pager')

In [ ]:
#dns client setup
stdout, stderr = N2.execute(f'git clone https://github.com/imgavinhi/RIT_Network_Research')

In [ ]:
#copying dns client/server files to where they should go
#copying dns client files to where they should go
stdout, stderr = N1.execute(f'sudo cp /home/ubuntu/RIT_Network_Research/Jupyter_Fabric_Testbed_Infrastructure/DNS_Infra/resolv.conf /etc/resolv.conf')
stdout, stderr = N2.execute(f'sudo cp /home/ubuntu/RIT_Network_Research/Jupyter_Fabric_Testbed_Infrastructure/DNS_Infra/resolv.conf /etc/resolv.conf')

In [ ]:
#need to install tshark on client and server (tshark)